Opstellen vergelijkingen

Ik begon met bereken hoeveel nieuw zout er bij komt. $0.2*6 = 1.2$ Voor vat x kan het volgende worden gesteld
De verandering van zout in vat \$x = nieuw zout + binnenkomend van vat y - zout dat naar vat y gaat - zout dat wegspoeld$


oftewel\
$x' = 1.2 + \frac{y}{100}- \frac{x}{100}*7$\
$y' = 3\frac{x}{100} - 3\frac{y}{100}$

Exacter\
$x'{n} = 1.2 + \frac{y_n}{100} - \frac{x_n}{100}*7$\
$x(0) = 0$ \
$x(n) = x_{n-1}+x'_n$ 



In [ ]:
def solve(endTime, step=1,
          In=6,
          XtY=3,
          Xout=4,
          YtX=1,
          Yout=2,
          volumeX=100,
          volumeY=100,
          saltCon=0.2,
          x=0,
          y=20):
    
    salt = lambda current, salt, volume : current * step * (salt / volume)
    derX = lambda x, y : (In * step * saltCon) + salt(YtX, y, volumeY) - salt(XtY + Xout, x, volumeX)
    derY = lambda x, y : salt(XtY, x, volumeY) - salt(YtX + Yout, y, volumeY)

    xFunc = lambda x, y : x + derX(x,y)
    yFunc = lambda x, y : y + derY(x,y)
    
    xValues = []
    yValues = []
    i = 0
    #Range doenst work with floats :(
    
    while i < endTime:
        xValues.append(x)
        yValues.append(y)
        x = xFunc(x,y)
        y = yFunc(x,y)
        i += step
    
    return xValues, yValues
        
       
       

In [ ]:
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import numpy as np

def plot_func(**kwargs):
    func = lambda x: float(x)
    sim_parameters = dict(zip(kwargs.keys(), map(func, kwargs.values())))
    step = sim_parameters["step"]
    beginRange = sim_parameters["startTime"]
    
    del sim_parameters["startTime"]
    x, y = solve(**sim_parameters)
    xAxis = np.array([i * step for i in range(len(x))])
    xAxisInRange = xAxis > beginRange
    x = np.array(x)
    y = np.array(y)
    x = x[xAxisInRange]
    y = y[xAxisInRange]
    
    xAxis = xAxis[xAxisInRange]
    plt.plot(x)
    plt.plot(y)
    tick, labels = plt.xticks()
    newLabel = lambda x : x * step + beginRange
    plt.xticks(tick, np.round(newLabel(tick)))
    plt.show

def create_UI(funcToPasVarsTo):
    larger = Layout(width='200%', height='80px')
    smaller = Layout(width='15%', height='80px')
    style = {'description_width': 'initial'}
    freq = widgets.FloatSlider(value=1, min=0.2, max=10.2, step=0.2 , description = "interval in which x and y are calculated min", style=style, layout=larger) 
    startTime =widgets.Text("0", description = "Start time", style=style, layout=smaller)
    endTime = widgets.Text("120", description = "End time", style=style, layout=smaller)
    In = widgets.Text("6", description = "Current entering x", style=style, layout=smaller)
    XtY = widgets.Text("3", description = "Current from x to y", style=style, layout=smaller)
    Xout = widgets.Text("4", description = "Current leaving x", style=style, layout=smaller)
    YtX = widgets.Text("1", description = "Current from y to x", style=style, layout=smaller)
    Yout = widgets.Text("2" , description = "Current leaving y", style=style, layout=smaller)
    volumeX = widgets.Text("100", description = "Volume of x in L", style=style, layout=smaller)
    volumeY = widgets.Text("100", description = "Volume of y in L", style=style, layout=smaller)
    saltCon = widgets.Text("0.2", description = "Consentraion of salt in kg/l", style=style)
    x = widgets.Text("0", description = "salt in x at t=0", style=style, layout=smaller)
    y = widgets.Text("20", description = "salt in y at t=0", style=style, layout=smaller)

    graphBox = widgets.HBox([freq])
    timeBox = widgets.HBox([startTime, endTime])
    currentBox =widgets.HBox([In, XtY, Xout, YtX, Yout])
    volumeBox = widgets.HBox([volumeX, volumeY])
    other = widgets.HBox([saltCon,x,y])
    mainBox =widgets.VBox([graphBox, timeBox, currentBox, volumeBox, other])

    out = widgets.interactive_output(funcToPasVarsTo, {'step': freq, 'startTime': startTime, 'endTime': endTime,
                                                 'In': In, 'XtY': XtY, 'Xout': Xout,
                                                 'YtX': YtX, 'Yout': Yout, 'volumeX': volumeX,
                                                 'volumeY': volumeY, 'saltCon': saltCon, 'x': x, "y": y})
    return mainBox, out

UI, output = create_UI(plot_func)
display(UI, output)
       